In [0]:
%sql
USE CATALOG catproject_catalog

In [0]:
%sql 
USE SCHEMA ecom_bronze

In [0]:
%sql
CREATE TABLE ecom_bronze.employee_raw (
  employee_id        STRING,
  full_name          STRING,
  date_of_birth      DATE,
  gender             STRING,
  email              STRING,
  phone_number       STRING,
  address            STRING,
  department         STRING,
  position           STRING,
  base_salary        DECIMAL(18,2),
  salary_coefficient DECIMAL(5,2),
  status             STRING,
  start_date         DATE,
  end_date           DATE,
  source_system      STRING,
  ingestion_time     TIMESTAMP,
  ingestion_year      INT,        -- Partition cấp 1
  ingestion_month     INT,        -- Partition cấp 2
  record_hash        STRING,
  operation_type     STRING,
  source_updated_at  TIMESTAMP
)
USING DELTA
PARTITIONED BY (ingestion_year, ingestion_month)

Tạo constraint cho các attribute để đảm bảo data integrity

In [0]:
%sql
ALTER TABLE ecom_bronze.employee_raw 
ALTER COLUMN employee_id SET NOT NULL;

ALTER TABLE ecom_bronze.employee_raw 
ADD CONSTRAINT gender_check CHECK (gender IN ('Male', 'Female'));

ALTER TABLE ecom_bronze.employee_raw 
ADD CONSTRAINT salary_check CHECK (base_salary >= 0);

ALTER TABLE ecom_bronze.employee_raw 
ADD CONSTRAINT operation_check CHECK (operation_type IN ('INSERT', 'UPDATE'));

ALTER TABLE ecom_bronze.employee_raw 
ADD CONSTRAINT birthday_check CHECK (date_of_birth <= current_date());

ALTER TABLE ecom_bronze.employee_raw 
ALTER COLUMN full_name SET NOT NULL;

ALTER TABLE ecom_bronze.employee_raw 
ADD CONSTRAINT salary_coefficient_check CHECK (salary_coefficient >= 0);

ALTER TABLE ecom_bronze.employee_raw
ADD CONSTRAINT valid_status 
CHECK (status IN ('active', 'suspended'));

ALTER TABLE ecom_bronze.employee_raw
ADD CONSTRAINT valid_start_date 
CHECK (start_date <= current_date());

ALTER TABLE ecom_bronze.employee_raw
ALTER COLUMN ingestion_time SET NOT NULL;

ALTER TABLE ecom_bronze.employee_raw
ADD CONSTRAINT valid_ingestion_time
CHECK (ingestion_time <= current_timestamp());


Tạo comment mô tả nội dung và thuộc tính của các cột trong bảng

In [0]:
%sql
-- COMMENT từng cột
COMMENT ON COLUMN ecom_bronze.employee_raw.employee_id          IS 'Mã định danh duy nhất của nhân viên trong hệ thống nguồn';
COMMENT ON COLUMN ecom_bronze.employee_raw.full_name            IS 'Họ và tên đầy đủ của nhân viên';
COMMENT ON COLUMN ecom_bronze.employee_raw.date_of_birth        IS 'Ngày sinh của nhân viên';
COMMENT ON COLUMN ecom_bronze.employee_raw.gender               IS 'Giới tính của nhân viên: Male, Female, Other';
COMMENT ON COLUMN ecom_bronze.employee_raw.email                IS 'Email công ty hoặc cá nhân của nhân viên';
COMMENT ON COLUMN ecom_bronze.employee_raw.phone_number         IS 'Số điện thoại liên hệ';
COMMENT ON COLUMN ecom_bronze.employee_raw.address              IS 'Địa chỉ hiện tại của nhân viên';
COMMENT ON COLUMN ecom_bronze.employee_raw.department           IS 'Tên phòng ban mà nhân viên thuộc về';
COMMENT ON COLUMN ecom_bronze.employee_raw.position             IS 'Chức danh hoặc vai trò của nhân viên';
COMMENT ON COLUMN ecom_bronze.employee_raw.base_salary          IS 'Mức lương cơ bản theo hợp đồng';
COMMENT ON COLUMN ecom_bronze.employee_raw.salary_coefficient   IS 'Hệ số điều chỉnh lương theo thâm niên, trình độ...';
COMMENT ON COLUMN ecom_bronze.employee_raw.status               IS 'Trạng thái làm việc: active, resigned, suspended';
COMMENT ON COLUMN ecom_bronze.employee_raw.start_date           IS 'Ngày bắt đầu làm việc chính thức';
COMMENT ON COLUMN ecom_bronze.employee_raw.end_date             IS 'Ngày nghỉ việc (nếu có)';
COMMENT ON COLUMN ecom_bronze.employee_raw.source_system        IS 'Hệ thống nguồn cung cấp dữ liệu (VD: HRM, ERP)';
COMMENT ON COLUMN ecom_bronze.employee_raw.ingestion_time       IS 'Thời điểm ingest bản ghi vào warehouse';
COMMENT ON COLUMN ecom_bronze.employee_raw.ingestion_year       IS 'Năm ingest dữ liệu (phục vụ partition)';
COMMENT ON COLUMN ecom_bronze.employee_raw.ingestion_month      IS 'Tháng ingest dữ liệu (phục vụ partition)';
COMMENT ON COLUMN ecom_bronze.employee_raw.record_hash          IS 'Hash checksum toàn bộ bản ghi, phục vụ so sánh thay đổi';
COMMENT ON COLUMN ecom_bronze.employee_raw.operation_type       IS 'Loại thay đổi: INSERT / UPDATE / DELETE';
COMMENT ON COLUMN ecom_bronze.employee_raw.source_updated_at    IS 'Thời điểm bản ghi được cập nhật tại hệ thống nguồn';


Kích hoạt CDF để theo dõi thay đổi dữ liệu vì bảng này thuộc SCD Type 2

In [0]:
%sql
ALTER TABLE ecom_bronze.employee_raw 
SET TBLPROPERTIES (
  delta.enableChangeDataFeed = true
);

In [0]:
%sql
DESCRIBE TABLE EXTENDED ecom_bronze.employee_raw